# OpenAI_API_KEY와 PROJECT_ID 가져오기

In [ ]:
import os

# .env에 저장해 놓은 openai_api_key와 project_id 변수로 가져오기
my_api_key = os.getenv("OPENAI_API_KEY")
my_project_id = os.getenv("PROJECT_ID")

## 더미 데이터 DataFrame으로 가져오기

In [ ]:
import pandas as pd

# CSV 파일을 데이터프레임으로 읽기
df_stores = pd.read_csv("stores.csv")
df_transactions = pd.read_csv("transactions.csv")

# 데이터 확인
display(df_stores.head())
display(df_transactions.head())

,store_id,상호명,주소,위도,경도,카테고리,세부카테고리,메뉴정보,영업시간,가게소개,주차가능여부,최대수용인원,편의시설,평점,리뷰수
0,1001,오브젝트 커피,서울 마포구 연남동 223-24,37.5602,126.9264,카페,로스터리카페,"[{""메뉴명"": ""아메리카노"", ""가격"": 4800}, {""메뉴명"": ""플랫화이트""...","{""평일"": ""09:00-22:00"", ""주말"": ""10:00-23:00""}","직접 로스팅한 스페셜티 원두를 사용하며, 감성적인 인테리어가 특징입니다. 혼자서 간...",공영주차장 이용,25,"[""와이파이"", ""콘센트"", ""바 테이블""]",4.6,254
1,1002,옛찻집 소담,서울 종로구 인사동길 42,37.5732,126.9851,카페,전통찻집,"[{""메뉴명"": ""쌍화차"", ""가격"": 7000}, {""메뉴명"": ""대추차"", ""가...","{""평일"": ""11:00-21:00"", ""주말"": ""11:00-22:00""}",고즈넉한 한옥에서 즐기는 건강한 전통차 전문점입니다. 단체석이 완비되어 있습니다.,불가,30,"[""단체석"", ""장애인 편의시설""]",4.8,312
2,1003,파파스 베이커리,서울 송파구 잠실동 188-1,37.5112,127.0823,베이커리,베이커리카페,"[{""메뉴명"": ""소금빵"", ""가격"": 3500}, {""메뉴명"": ""딸기케이크"", ...","{""평일"": ""08:00-21:00"", ""주말"": ""08:00-22:00""}",유기농 밀가루와 신선한 재료로 매일 빵을 굽습니다. 아이들과 함께 가기 좋은 식당으...,가능 (5대),40,"[""와이파이"", ""유아의자"", ""놀이방"", ""수유실""]",4.7,451
3,1004,농부의 마음,서울 서초구 서초대로 321,37.4923,127.0088,우리 농산물,산지직송,"[{""메뉴명"": ""유기농 상추 (1봉)"", ""가격"": 3000}, {""메뉴명"": ""...","{""평일"": ""10:00-20:00"", ""주말"": ""휴무""}",농가에서 당일 수확한 신선한 채소와 과일을 판매합니다.,가능 (2대),10,"[""장애인 편의시설""]",4.9,88
4,1005,리틀 이탈리아,서울 강남구 강남대로 456,37.5023,127.0245,양식,이탈리안,"[{""메뉴명"": ""봉골레 파스타"", ""가격"": 16000}, {""메뉴명"": ""마르게...","{""평일"": ""11:30-22:00"", ""주말"": ""11:30-23:00""}",애견 동반이 가능한 아늑한 분위기의 이탈리안 레스토랑입니다.,발렛파킹,35,"[""애견동반"", ""와이파이"", ""예약가능""]",4.5,560


,order_id,store_id,주문일시,요일,날씨,메뉴명,수량,가격,메뉴원가,주문유형,주변이벤트,고객ID,프로모션정보
0,2025091901,1001,2025-09-19 13:05:12,금요일,비,아메리카노,2,9600,1920,매장,없음,user_101,없음
1,2025091901,1001,2025-09-19 13:05:12,금요일,비,플랫화이트,1,5500,1650,매장,없음,user_101,없음
2,2025091902,1001,2025-09-19 14:20:01,금요일,비,아메리카노,1,4800,960,포장,없음,user_102,없음
3,2025092001,1001,2025-09-20 12:30:00,토요일,맑음,아메리카노,2,9600,1920,매장,없음,user_104,주말할인
4,2025092101,1001,2025-09-21 15:00:00,일요일,맑음,플랫화이트,4,22000,6600,포장,없음,user_106,주말할인


In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from pandasai import SmartDataframe
from pandasai.llm import OpenAI

# --- 1. 설정 단계 ---
print("="*60)
print("NH 로컬오더 AI 챗봇 시연 프로그램을 시작합니다.")
print("="*60)

# .env 파일에서 환경변수를 불러옵니다. (API 키/프로젝트 ID 은닉)
load_dotenv()

# 환경변수에서 키와 ID를 안전하게 가져옵니다.
my_api_key = os.getenv("OPENAI_API_KEY")
my_project_id = os.getenv("PROJECT_ID")

# 환경변수가 제대로 로드되었는지 확인합니다.
if not my_api_key or not my_project_id:
    raise ValueError("환경변수에서 API 키 또는 프로젝트 ID를 찾을 수 없습니다. .env 파일을 확인하세요.")
print("✅ API 키 및 프로젝트 ID 로딩 완료")

# --- 2. 데이터 준비 단계 ---
try:
    # CSV 파일들을 각각의 데이터프레임으로 읽어옵니다.
    df_stores = pd.read_csv("stores.csv")
    df_transactions = pd.read_csv("transactions.csv")
    print("✅ 매장 및 거래 데이터 로딩 완료")
    print(f"   - 총 매장 데이터: {len(df_stores)}건")
    print(f"   - 총 거래 데이터: {len(df_transactions)}건")
except FileNotFoundError as e:
    raise FileNotFoundError(f"CSV 파일을 찾을 수 없습니다: {e}. 스크립트와 동일한 위치에 파일이 있는지 확인하세요.")

# --- 3. AI 모델 설정 단계 ---
# OpenAI LLM 클라이언트를 설정합니다. 이 llm 객체는 두 챗봇이 함께 사용합니다.
llm = OpenAI(
    api_token=my_api_key,
    default_headers={
        "OpenAI-Project": my_project_id
    }
)
print("✅ OpenAI LLM 클라이언트 설정 완료")
print("\n")


# --- 4. 소비자용 AI 챗봇 시연 ---
print("="*60)
print("👤 소비자용 AI 챗봇 시연 시작")
print("="*60)

# 'stores.csv' 데이터를 기반으로 하는 SmartDataframe을 생성합니다.
sdf_consumer = SmartDataframe(df_stores, config={"llm": llm})

# 시연할 질문 리스트
consumer_prompts = [
    "지금 강원도 소노펠리체 델피노 가는 길인데 7명이 먹기 좋은 가장 가까운 카페 알려줘",
    "주변에 주차되는 음식점 알려줘",
    "혼자서 간단히 먹을 수 있는 곳 추천해줘",
    "아이들과 함께 가기 좋은 식당이 있을까?",
    "운영 시간이 가장 긴 카페는 뭐야? 한줄로 알려줘 문장처럼 대답해줘"
]

# 각 질문에 대한 답변을 순서대로 출력합니다.
for prompt in consumer_prompts:
    print(f"❓ 사용자 질문 > {prompt}")
    response = sdf_consumer.chat(prompt)
    print(f"🤖 AI 답변 > {response}")
    print("-" * 30)

print("\n")

# --- 5. 업주용 AI 챗봇 시연 ---
print("="*60)
print("💼 업주용 AI 챗봇 시연 시작")
print("="*60)

# 'transactions.csv' 데이터를 기반으로 하는 SmartDataframe을 생성합니다.
sdf_owner = SmartDataframe(df_transactions, config={"llm": llm})

# 시연할 질문 리스트
owner_prompts = [
    "비가 오는 금요일에 가장 인기 있을 메뉴와 예상 매출액을 알려줘",
    "최근 가장 실적이 저조한 메뉴는 뭐야? 단종을 고려해야 할까?",
    "객단가(1인당 평균 구매 금액)를 가장 효과적으로 높일 수 있는 메뉴 조합을 추천해 줘",
    "이번 주말 프로모션 아이디어를 제안해줘"
]

# 각 질문에 대한 답변을 순서대로 출력합니다.
for prompt in owner_prompts:
    print(f"❓ 업주 질문 > {prompt}")
    response = sdf_owner.chat(prompt)
    print(f"🤖 AI 답변 > {response}")
    print("-" * 30)

print("\n")
print("="*60)
print("모든 시연이 성공적으로 종료되었습니다.")
print("="*60)

NH 로컬오더 AI 챗봇 시연 프로그램을 시작합니다.
✅ API 키 및 프로젝트 ID 로딩 완료
✅ 매장 및 거래 데이터 로딩 완료
   - 총 매장 데이터: 30건
   - 총 거래 데이터: 16건
✅ OpenAI LLM 클라이언트 설정 완료


👤 소비자용 AI 챗봇 시연 시작
❓ 사용자 질문 > 지금 강원도 소노펠리체 델피노 가는 길인데 7명이 먹기 좋은 가장 가까운 카페 알려줘
{'type': 'string', 'value': 'The closest cafe is 강원도 막국수 located at 강원 속초시 중앙로 129.'}
🤖 AI 답변 > The closest cafe is 강원도 막국수 located at 강원 속초시 중앙로 129.
------------------------------
❓ 사용자 질문 > 주변에 주차되는 음식점 알려줘
{'type': 'dataframe', 'value':     store_id        상호명                   주소       위도        경도    카테고리  \
2       1003   파파스 베이커리     서울 송파구 잠실동 188-1  37.5112  127.0823    베이커리   
3       1004     농부의 마음      서울 서초구 서초대로 321  37.4923  127.0088  우리 농산물   
8       1009    강원도 막국수       강원 속초시 중앙로 129  38.2049  128.5911      한식   
9       1010       바다정원  강원 고성군 토성면 아야진리 242  38.2714  128.5302      카페   
13      1014   제주은희네해장국      서울 용산구 한강대로 288  37.5342  126.9729      한식   
15      1016        산지애      서울 송파구 양재대로 932  37.4981  127.1245  우리 농산물   
23      1024     

Traceback (most recent call last):
  File "c:\Users\edukd\miniconda3\envs\NH-venv\Lib\site-packages\pandasai\pipelines\chat\generate_chat_pipeline.py", line 335, in run
    ).run(input)
      ^^^^^^^^^^
  File "c:\Users\edukd\miniconda3\envs\NH-venv\Lib\site-packages\pandasai\pipelines\pipeline.py", line 137, in run
    raise e
  File "c:\Users\edukd\miniconda3\envs\NH-venv\Lib\site-packages\pandasai\pipelines\pipeline.py", line 101, in run
    step_output = logic.execute(
                  ^^^^^^^^^^^^^^
  File "c:\Users\edukd\miniconda3\envs\NH-venv\Lib\site-packages\pandasai\pipelines\chat\code_execution.py", line 113, in execute
    raise e
  File "c:\Users\edukd\miniconda3\envs\NH-venv\Lib\site-packages\pandasai\pipelines\chat\code_execution.py", line 85, in execute
    result = self.execute_code(code_to_run, code_context)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\edukd\miniconda3\envs\NH-venv\Lib\site-packages\pandasai\pipelines\chat\code_executio